In [3]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division

In [4]:
# reqs
fqdir = '/home/lindb/scratch/seqcap_pine'
shdir = op.join(fqdir,'shfiles')
lensh = 500                               # number of sh files that can be qsubbed
cd(fqdir)

In [15]:
import sys
"/".join([x for x in sys.argv[0].split("/")[:4]])

'/global/home/lindb'

In [5]:
gzfiles = !ls *.gz
gzfiles = [f for f in gzfiles if '.Pi_' in f and 'R1' in f]
len(gzfiles),sorted(gzfiles)

(189,
 ['paired_HI.0748.006.Index_1.Pi_64A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_10.Pi_277A3_R1.fastq.gz',
  'paired_HI.0748.006.Index_11.Pi_129A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_12.Pi_131A3_R1.fastq.gz',
  'paired_HI.0748.006.Index_2.Pi_156A3_R1.fastq.gz',
  'paired_HI.0748.006.Index_3.Pi_70A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_4.Pi_120A3_R1.fastq.gz',
  'paired_HI.0748.006.Index_5.Pi_161A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_6.Pi_128A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_7.Pi_165A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_8.Pi_170A4_R1.fastq.gz',
  'paired_HI.0748.006.Index_9.Pi_36A3_R1.fastq.gz',
  'paired_HI.0931.001.BioO_1.Pi_167A4_R1.fastq.gz',
  'paired_HI.0931.001.BioO_2.Pi_281A4_R1.fastq.gz',
  'paired_HI.0931.001.BioO_3.Pi_87A5_R1.fastq.gz',
  'paired_HI.0931.001.BioO_4.Pi_259A3_R1.fastq.gz',
  'paired_HI.0931.001.BioO_5.Pi_80A3_R1.fastq.gz',
  'paired_HI.0931.001.BioO_6.Pi_2A3_R1.fastq.gz',
  'paired_HI.0941.004.BioO_10.Pi_96A4_R1.fastq.gz'

In [10]:
# match seq pairs, alert if pair not found
seq_pairs = []
for f in gzfiles:
    assert 'R1' in f
    read2 = f.replace("_R1","_R2")
    if op.exists(read2):
        seq_pairs.append((op.abspath(f),op.abspath(read2)))
    else:
        !echo 'no pair for '$f >> $fqdir'/messages/msgs.txt'
len(seq_pairs)

187

In [60]:
seq_pairs = [('/home/lindb/scratch/seqcap_pine/paired_HI.0941.007.BioO_4.Pi_53A3_R1.fastq.gz',
             '/home/lindb/scratch/seqcap_pine/paired_HI.0941.007.BioO_4.Pi_53A3_R2.fastq.gz'
            )]

In [11]:
# make some dirs
shtrimDIR    = op.join(shdir,'trimmed_shfiles')
trimDIR      = op.join(fqdir,'trimmed')         # outfiles
for d in [shtrimDIR,trimDIR]:
    if not op.exists(d):
        os.makedirs(d)

# determine how many commands per sh file
if len(seq_pairs) <= lensh:
    # one command per sh file
    ceil = 1
else:
    # multiple commands per sh file
    ceil = math.ceil(len(seq_pairs)/lensh)

shcount = 0
fcount  = 0
tcount  = 0
text    = ''''''
for s in seq_pairs:
    r1    = op.abspath(s[0])
    r1out = op.join(trimDIR,op.basename(r1).replace('.fastq.gz','_trimmedandfiltered.fastq.gz'))
    r2    = op.abspath(s[1])
    r2out = op.join(trimDIR,op.basename(r2).replace('.fastq.gz','_trimmedandfiltered.fastq.gz'))
    cmd   = '''fastp -i %s -o %s \
-I %s -O %s \
-g \
--cut_window_size 5 \
--cut_mean_quality 30 \
--qualified_quality_phred 30 \
--unqualified_percent_limit 20 \
--n_base_limit 5 \
--length_required 75 \
-h "%s.html" \
--cut_by_quality3 \
--thread 1 \
--json "%s.json"

''' % (r1,
       r1out,
       r2,
       r2out,
       r1out.replace("_R1_trimmed.fastq.gz","_stats.html"),
       r1out.replace("_R1_trimmed.fastq.gz",".json")
      )
    text = text + cmd
    
    fcount += 1
    tcount += 1
    if fcount == ceil or tcount == len(seq_pairs):
        shz = str(shcount).zfill(3)
        header = '''#!/bin/bash
#PBS -N trim%s
#PBS -V
#PBS -S /bin/bash
#PBS -l nodes=1:ppn=1
#PBS -l procs=1
#PBS -l walltime=0:10:00
#PBS -l mem=600mb
''' % (shz)
        text = header + text
        filE = op.join(shtrimDIR,'trim_%s.sh' % str(shcount).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        shcount += 1
        fcount = 0
        text = ''''''

('/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_10.Pi_277A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_10.Pi_277A3_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_11.Pi_129A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_11.Pi_129A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_12.Pi_131A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_12.Pi_131A3_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_1.Pi_64A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_1.Pi_64A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_2.Pi_156A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_2.Pi_156A3_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.Index_3.Pi_70A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0748.006.In

('/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_6.Pi_166A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_6.Pi_166A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_7.Pi_140A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_7.Pi_140A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_8.Pi_582A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_8.Pi_582A3_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_9.Pi_150A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.001.BioO_9.Pi_150A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.002.BioO_10.Pi_210A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.002.BioO_10.Pi_210A3_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.002.BioO_11.Pi_57A5_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.002.BioO_11.Pi_57A

('/global/scratch/lindb/seqcap_pine/paired_HI.0942.007.BioO_7.Pi_198A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.007.BioO_7.Pi_198A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.007.BioO_8.Pi_163A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.007.BioO_8.Pi_163A3_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.007.BioO_9.Pi_242A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.007.BioO_9.Pi_242A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.008.BioO_10.Pi_35A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.008.BioO_10.Pi_35A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.008.BioO_11.Pi_37A4_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.008.BioO_11.Pi_37A4_R2.fastq.gz')
('/global/scratch/lindb/seqcap_pine/paired_HI.0942.008.BioO_12.Pi_255A3_R1.fastq.gz', '/global/scratch/lindb/seqcap_pine/paired_HI.0942.008.BioO_12.Pi_255A

In [ ]:
shs = fs(shtrimDIR)
for f in shs:
    !qsub $f